In [1]:
sample_image_path = "/mnt/c/Users/kmitchell/Documents/GitHub/Thesis/input_data/rgb_test/2021_GRSM_5_267000_3942000_image.tif"
from deepforest import main
from deepforest import get_data
from deepforest import utilities
import matplotlib.pyplot as plt
import rasterio
import rasterio.features
import rasterio.warp
import os
model = main.deepforest()
model.use_release()

Reading config file: /home/kmitchell/miniconda3/envs/micropheno/lib/python3.11/site-packages/deepforest/data/deepforest_config.yml


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


No validation file provided. Turning off validation loop


/home/kmitchell/miniconda3/envs/micropheno/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


Model from DeepForest release https://github.com/weecology/DeepForest/releases/tag/1.0.0 was already downloaded. Loading model from file.
Loading pre-built model: https://github.com/weecology/DeepForest/releases/tag/1.0.0


In [13]:


model = main.deepforest()
model.use_release()

sample_image = get_data(sample_image_path)

img = model.predict_tile(sample_image, return_plot = True, patch_size=500,patch_overlap=0.25)
plt.imsave(f"output3_patch_size=500_patch_overlap=0.25.png", img[:,:,::-1])

#predict_image returns plot in BlueGreenRed (opencv style), but matplotlib likes RedGreenBlue, switch the channel order. Many functions in deepforest will automatically perform this flip for you and give a warning.
# plt.imshow(img[:,:,::-1])
# plt.imsave(f"output2_patch_size=500_patch_overlap=0.1.png", img[:,:,::-1])

Reading config file: /home/kmitchell/miniconda3/envs/micropheno/lib/python3.11/site-packages/deepforest/data/deepforest_config.yml


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


No validation file provided. Turning off validation loop
Model from DeepForest release https://github.com/weecology/DeepForest/releases/tag/1.0.0 was already downloaded. Loading model from file.
Loading pre-built model: https://github.com/weecology/DeepForest/releases/tag/1.0.0


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 729/729 [00:15<00:00, 47.53it/s]
20346 predictions in overlapping windows, applying non-max supression
11628 predictions kept after non-max suppression


/home/kmitchell/miniconda3/envs/micropheno/lib/python3.11/site-packages/deepforest/visualize.py:118: UserWarning: Input images must be channels last format [h, w, 3] not channels first [3, h, w], using np.rollaxis(image, 0, 3) to invert!
  warnings.warn("Input images must be channels last format [h, w, 3] not channels "
/home/kmitchell/miniconda3/envs/micropheno/lib/python3.11/site-packages/deepforest/visualize.py:126: UserWarning: No color was provided and the label column is not numeric. Using a single default color.
  warnings.warn("No color was provided and the label column is not numeric. "


In [3]:
sample_image_path = "/mnt/c/Users/kmitchell/Documents/GitHub/Thesis/input_data/rgb_test/2021_GRSM_5_267000_3942000_image.tif"

sample_image = get_data(sample_image_path)

predictions = model.predict_tile(sample_image, patch_size=500,patch_overlap=0.25)

with rasterio.open(sample_image_path) as df:
    transform = df.transform
    crs = df.crs
    gdf = utilities.annotations_to_shapefile(predictions, transform=transform, crs=crs)
    print(gdf.total_bounds)
    print(gdf.crs)
    # CRS.from_epsg(32617)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/kmitchell/miniconda3/envs/micropheno/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 729/729 [00:14<00:00, 50.76it/s]
20346 predictions in overlapping windows, applying non-max supression
11628 predictions kept after non-max suppression
[ 267294.65 3941999.95  268000.05 3942999.95]
PROJCS["WGS 84 / UTM zone 17N",GEOGCS["WGS 84",DATUM["World Geodetic System 1984",SPHEROID["WGS 84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-81],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


/home/kmitchell/miniconda3/envs/micropheno/lib/python3.11/site-packages/deepforest/utilities.py:476: UserWarning: This method is deprecated and will be removed in version DeepForest 2.0.0, please use boxes_to_shapefile which unifies project_boxes and annotations_to_shapefile functionalities
  warnings.warn("This method is deprecated and will be "


In [48]:

output_dir = "bounding_box_images"
os.makedirs(output_dir, exist_ok=True)
# Sort the dataframe by score and filter to top 500
gdf = gdf.sort_values(by="score", ascending=False)
gdf = gdf.head(500)


# Open the original image
with rasterio.open(sample_image_path) as src:
    # Loop through each bounding box in the gdf dataframe
    for index, row in gdf.iterrows():
        
        # Get the score of the bounding box
        score = row['score']
        # Get the bounding box coordinates
        xmin, ymin, xmax, ymax = row['xmin'], row['ymin'], row['xmax'], row['ymax']

        # Create a rasterio window for the bounding box
        window=rasterio.windows.Window(xmin, ymin, xmax - xmin, ymax - ymin)

        # Read the subset of the image within the bounding box
        subset = src.read(window=window)



        # Create a new rasterio dataset for the subset
        subset_dataset = rasterio.open(
            f"{output_dir}/bounding_box_{index}_score={score}.tif",
            'w',
            driver='GTiff',
            height=subset.shape[1],
            width=subset.shape[2],
            count=subset.shape[0],
            dtype=subset.dtype,
            crs=src.crs,
            transform=src.window_transform(window)
        )
        
        # Write the subset to the new dataset
        subset_dataset.write(subset)
        subset_dataset.close()
        


In [1]:
from bbox import Image
sample_image_path = "/mnt/c/Users/kmitchell/Documents/GitHub/Thesis/input_data/rgb_test/2021_GRSM_5_267000_3942000_image.tif"

In [2]:
img = Image(sample_image_path)

Reading config file: /home/kmitchell/miniconda3/envs/micropheno/lib/python3.11/site-packages/deepforest/data/deepforest_config.yml


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


No validation file provided. Turning off validation loop


/home/kmitchell/miniconda3/envs/micropheno/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


Model from DeepForest release https://github.com/weecology/DeepForest/releases/tag/1.0.0 was already downloaded. Loading model from file.
Loading pre-built model: https://github.com/weecology/DeepForest/releases/tag/1.0.0


In [3]:
img.get_bounding_boxes()

: 

In [16]:
img.get_bounding_boxes()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/kmitchell/miniconda3/envs/micropheno/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 729/729 [00:13<00:00, 54.74it/s]
0 predictions in overlapping windows, applying non-max supression
0 predictions kept after non-max suppression


/home/kmitchell/miniconda3/envs/micropheno/lib/python3.11/site-packages/deepforest/utilities.py:476: UserWarning: This method is deprecated and will be removed in version DeepForest 2.0.0, please use boxes_to_shapefile which unifies project_boxes and annotations_to_shapefile functionalities
  warnings.warn("This method is deprecated and will be "
